## Capstone Project- Opening a shopping mall in Mumbai, India

<h2>Steps:</h2>

1. Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page <br /> 
2. Get the geographical coordinates of the neighborhoods <br /> 
3. Obtain the venue data for the neighborhoods from Foursquare API <br /> 
4. Explore and cluster the neighborhoods <br /> 
5. Select the best cluster to open a new shopping mall

<h3>1. Import Libraries</h3>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

! pip install geopy
! pip install geocoder
! pip install BeautifulSoup4

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 122kB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 19.1MB/s ta 0:00:01
     |████████████████████████████████| 92kB 19.9MB/s eta 0:00:01
     |████████████████████████████████| 829kB 25.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |████████████████████████████████| 122kB 7.2MB/s eta 0:00:01
Libraries imported.


<h3>2. Scrape data from Wikipedia </h3>

In [4]:
# send the GET request
url="https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai"
data = requests.get(url).text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list --- class can be found by going to page source of the wikipedia page
for row in soup.find_all("div", class_="toc")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,1 Western Suburbs\n\n1.1 Andheri\n1.2 Mira-Bha...
1,1.1 Andheri
2,1.2 Mira-Bhayandar
3,1.3 Bandra
4,1.4 Borivali


In [9]:
kl_df.shape

(45, 1)

Cleaning the Data

In [10]:
#removing the first row as it has concatenated string
df = kl_df.iloc[1:43]
df.shape
df.head()

,Neighborhood
1,1.1 Andheri
2,1.2 Mira-Bhayandar
3,1.3 Bandra
4,1.4 Borivali
5,1.5 Dahisar


In [11]:
cleaned_df=df['Neighborhood'].str.split(' ').str[1]
#add a column name 
cleaned_df.columns=['Neighborhood']
#Convert back the object to Data Frame
df2=pd.DataFrame(cleaned_df)
df2.head()

,Neighborhood
1,Andheri
2,Mira-Bhayandar
3,Bandra
4,Borivali
5,Dahisar


<h3>3. Get the geographical Coordinates </h3>

In [12]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
#Created a dataframe Manually as geocoder call function is not getting any values for Lat and long
# as per the tutorial  it could give error sometimes and in that case we can get lat and long from some other source

lat_long_data=[['Andheri',19.1136,72.8697],['Mira-Bhayandar',19.295233,72.854393],['Bandra',19.0596,72.8295],['Borivali',19.2307,72.8567],['Dahisar',19.4946,72.8604],['Goregaon',19.1663,72.8526],['Jogeshwari',19.1439,72.8428],['Juhu',19.1075,72.8263],['Kandivali',19.2029,72.8518],['Khar',19.07,72.8338],['Malad',19.1874,72.8484],['Santacruz',19.0843,72.836],['Vasai',19.3919,72.8397],['Virar',19.4564,72.7925],['Vile',19.0968,72.8517],['Eastern',19.0968,72.8517],['Bhandup',19.1511,72.9372],['Ghatkopar',19.079,72.908],['Kanjurmarg',19.1294,72.933],['Kurla',19.0726,72.8845],['Mulund',19.1726,72.9425],['Powai',19.1176,72.906],['Vidyavihar',19.0798,72.8973],['Vikhroli',19.1142,72.9358],['Harbour',19.033,73.0297],['Chembur',19.032801,72.896355],['Govandi',19.0535,72.9239],['Mankhurd',19.0513,72.9368],['Trombay',19.0093,72.8984],['Antop',19.0283,72.8698],['Byculla',18.975,72.8295],['Colaba',18.9067,72.8147],['Dadar',19.0178,72.8478],['Girgaon',18.9572,72.8197],['Kalbadevi',18.9487,72.8289],['Kamathipura',18.9649,72.8261],['Matunga',19.0269,72.8553],['Parel',18.9977,72.8376],['Tardeo',18.9676,72.8141]]

In [14]:
df3 = pd.DataFrame(lat_long_data, columns = ['neighborhood','Latitude', 'Longitude'])

In [15]:
df3.head()

,neighborhood,Latitude,Longitude
0,Andheri,19.113600,72.869700
1,Mira-Bhayandar,19.295233,72.854393
2,Bandra,19.059600,72.829500
3,Borivali,19.230700,72.856700
4,Dahisar,19.494600,72.860400


<h3>4. Create a map of Mumbai, India </h3>

In [16]:

# get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India are 18.9387711, 72.8353355.


In [17]:
#Create a map

# create map of Mumbai using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

<h3>5. Use Foursquare API to explore neighborhoods </h3>

In [22]:

# define Foursquare Credentials and Version
CLIENT_ID = 'M2AKPDJGS2YJYWNLHCM0YL2H31MF1VWJPDGIUWBSY0MDD0DD' # your Foursquare ID
CLIENT_SECRET = 'MINZO4KMUQ4TX4WOZ1C1D22FCU5MIGLWLUY1TT1BEIIXHVCH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M2AKPDJGS2YJYWNLHCM0YL2H31MF1VWJPDGIUWBSY0MDD0DD
CLIENT_SECRET:MINZO4KMUQ4TX4WOZ1C1D22FCU5MIGLWLUY1TT1BEIIXHVCH


<h4>Let's get top venues in neighborhood </h4>

In [23]:
radius = 1500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [ ]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

Let's check how many venues are returned for each neighborhood

In [ ]:
venues_df.groupby(["Neighborhood"]).count()

In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

In [ ]:
# print out the list of categories -- to find out one having shopping mall
venues_df['VenueCategory'].unique()[:50]

In [ ]:
# check if the results contain "Shopping Mall"
"Mall" in venues_df['VenueCategory'].unique()

<h3>6. Analyse each neighborhood </h3>

In [ ]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

In [79]:
#Group by neighborhoods 
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(39, 195)


,Neighborhoods,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lingerie Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Resort,Restaurant,River,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Whisky Bar,Wine Bar,Women's Store,Zoo
0,Andheri,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.060000,0.00,0.010000,0.0,0.000000,0.000000,0.010000,0.050000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.150000,0.030000,0.010000,0.190000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.00,0.00000,0.010000,0.00,0.040000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.020000,0.00,0.00000,0.00,0.000000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.040000,0.0000,0.000000,0.000000,0.020000,0.010000,0.0000,0.000000,0.000000,0.01,0.050000,0.

In [20]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

NameError: name 'kl_grouped' is not defined

In [82]:
#Create a dataframe for Shopping mall only
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Antop,0.000000
2,Bandra,0.000000
3,Bhandup,0.038462
4,Borivali,0.000000


<h3>7. cluster the neighbors </h3>

In [85]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 1, 0, 0, 0, 1, 2, 0], dtype=int32)

In [86]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [87]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.000000,0
1,Antop,0.000000,0
2,Bandra,0.000000,0
3,Bhandup,0.038462,1
4,Borivali,0.000000,0


In [92]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = pd.merge(left=df3, right=kl_merged, left_on='neighborhood', right_on='Neighborhood')

print(kl_merged.shape)
kl_merged.head() 

(39, 6)


,neighborhood,Latitude,Longitude,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,19.113600,72.869700,Andheri,0.0,0
1,Mira-Bhayandar,19.295233,72.854393,Mira-Bhayandar,0.0,0
2,Bandra,19.059600,72.829500,Bandra,0.0,0
3,Borivali,19.230700,72.856700,Borivali,0.0,0
4,Dahisar,19.494600,72.860400,Dahisar,0.0,0


In [93]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(39, 6)


,neighborhood,Latitude,Longitude,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,19.113600,72.869700,Andheri,0.000000,0
35,Kamathipura,18.964900,72.826100,Kamathipura,0.000000,0
34,Kalbadevi,18.948700,72.828900,Kalbadevi,0.000000,0
33,Girgaon,18.957200,72.819700,Girgaon,0.000000,0
30,Byculla,18.975000,72.829500,Byculla,0.000000,0
29,Antop,19.028300,72.869800,Antop,0.000000,0
28,Trombay,19.009300,72.898400,Trombay,0.000000,0
27,Mankhurd,19.051300,72.936800,Mankhurd,0.000000,0
26,Govandi,19.053500,72.923900,Govandi,0.000000,0
25,Chembur,19.032801,72.896355,Chembur,0.000000,0


In [3]:
#visualize
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'latitude' is not defined

In [96]:
kl_merged.groupby(["Cluster Labels"]).count()

,neighborhood,Latitude,Longitude,Neighborhood,Shopping Mall
Cluster Labels,,,,,
0,23,23,23,23,23
1,9,9,9,9,9
2,7,7,7,7,7


<h2>Results </h2>

Most of the shopping malls are present in Cluster 0 which is a main region of the city Mumbai. This is due to high income individuals are more in number in these areas. Along with that, these areas have grown substantially in recent years in terms of development. 

Cluster 1 and Cluster 2 have fewer malls as compared to Cluster 1. This presents a good opportuntity for opening a new mall as there will be lesser competition and more scope. 
Property developers can choose one of these places to capatilize the advantage of first mover in these areas. The land prices in these areas will also be lower at this time. 

All in all, this project is based on the location data and current shopping mall locations only. Property owners can also analysis demogrpahics before coming to final conclusion